In [ ]:
from keras.datasets import reuters
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Lambda
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

max_features = 15000
text_max_words = 120

# 1. 데이터셋 생성하기

# 훈련셋과 시험셋 불러오기
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_features)

# 훈련셋과 검증셋 분리

x_train = x_train[:10]

# 데이터셋 전처리 : 문장 길이 맞추기
x_train = sequence.pad_sequences(x_train, maxlen=text_max_words)

In [ ]:
import keras.backend as K
import math

def pairing(input2):
    newinput=[]
    for i in range(10-2):
        for k in range(i, 10-2):
            if not i==k:
                newinput.append((cos_distance(input2[i], input2[k])+cos_distance(input2[i+2], input2[k+2]))/2)

    return K.variable(newinput)
    

def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)


def cos_distance(a, b):
    def l2_normalize(x, axis):
        norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=True))
        return K.maximum(x, K.epsilon()) / K.maximum(norm, K.epsilon())
    a = l2_normalize(a, axis=-1)
    b = l2_normalize(b, axis=-1)
    return -K.mean(a *b, axis=-1)
    

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=text_max_words))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Lambda(pairing))
# model.add(Lambda(make_side_cosine_similarity))

In [ ]:
model.summary()